In [13]:
#Author: Yitao Wu
#coding challange
#Clerkie

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import json
import datetime
import numpy as np
import pandas as pd
import time
stemmer = SnowballStemmer("english")



In [14]:
df = pd.read_csv('clerkie_questions.csv', names = ["class","sentence"])
training_data = df.values
data = df[['sentence']].values
label = df[['class']].values


In [15]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)

In [16]:
#training_data = np.concatenate((y_train, X_train), axis=1)
words = []
classes = []
documents = []
ignore_words = set(stopwords.words('english'))
ignore_words.add('?')
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern[1])
    # add to words list
    words.extend(w)
    # add to documents in corpus
    documents.append((w, pattern[0]))
    # add to classes list
    if pattern[0] not in classes:
        classes.append(pattern[0])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words and not w[0].isdigit()]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

In [17]:
# create training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)


In [18]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words



# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is the bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

# error threshold
ERROR_THRESHOLD = 0.6
def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ('[class, %]:',return_results[0])
    return return_results



In [19]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # loss
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # Break if this 10k iteration's error is greater than the last iteration
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)
        layer_1_error = layer_2_delta.dot(synapse_1.T)
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [20]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 60x69    Output matrix: 1x3
delta after 10000 iterations:0.013574668778778764
delta after 20000 iterations:0.012779965815814065
delta after 30000 iterations:0.012444433909432487
delta after 40000 iterations:0.012249482842559199
delta after 50000 iterations:0.012118711994035936
delta after 60000 iterations:0.012023409588991838
delta after 70000 iterations:0.011950086805201085
delta after 80000 iterations:0.011891473178140958
delta after 90000 iterations:0.011843263497382168
delta after 100000 iterations:0.01180272703637566
saved synapses to: synapses.json
processing time: 15.739877939224243 seconds


In [37]:
# load calculated words classes and vector values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    words = np.asarray(synapse['words'])
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])
    classes = np.asarray(synapse['classes'])
word_token = set(words)

In [43]:
y_pred = []
for i in X_test:
    y_pred.append(classify(i[0])[0][0])

[class, %]: [2, 0.9999628887292634]
[class, %]: [2, 0.9977301649978768]
[class, %]: [2, 0.8918495184165586]
[class, %]: [0, 0.9991419482888458]
[class, %]: [2, 0.961268612579491]
[class, %]: [2, 0.9994615865099298]
[class, %]: [0, 0.9998727435261083]
[class, %]: [0, 0.9989729855771858]
[class, %]: [1, 0.9998940198818518]
[class, %]: [0, 0.9857809250536024]
[class, %]: [1, 0.9970878311317071]
[class, %]: [2, 0.9986683268893317]


In [48]:
test_acc = accuracy_score(y_test, y_pred)

In [50]:
test_acc

1.0